In [7]:
sbb_oneday = sbb_inradius.filter("betriebstag == '14.05.2019'")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
timeDiff = (F.unix_timestamp('arrival_time_real', format="HH:mm:ss")
            - F.unix_timestamp('arrival_time_schedule', format="HH:mm"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
stcweekdays = spark.read.parquet("user/boecuego/stcweekdays.parquet")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
sbb_oneday_english = sbb_oneday_english.withColumn("arrival_time",\
                                                  sbb_oneday_english.schedule_arrival.substr(12,10000))
sbb_oneday_english = sbb_oneday_english.withColumn("departure_time",\
                                                  sbb_oneday_english.schedule_dep.substr(12,10000))

sbb_oneday_english_h = sbb_oneday_english.filter(" CAST(SUBSTRING(arrival_time,1,2) AS int) BETWEEN 7 AND 21 ")
sbb_oneday_english_h = sbb_oneday_english_h.filter(" CAST(SUBSTRING(departure_time,1,2) AS int) BETWEEN 7 AND 21 ")




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
stcweekdays = stcweekdays.withColumn("departure_time",\
                                                  stcweekdays.departure_time.substr(1,5))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
@F.udf(T.StringType())
def splt(x):
    return x.split(":")[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
def collect_stopss(sbb_oneday_english_h):
    sbb_collected_stops = sbb_oneday_english_h.groupby("trip_id")\
            .agg(F.concat_ws("-", F.collect_list(sbb_oneday_english_h.stop_id)).alias("stop_ids")\
            ,F.concat_ws("-", F.collect_list(sbb_oneday_english_h.departure_time)).alias("departures")).sort("stop_ids")
    return sbb_collected_stops


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
sbb_collected_stops = collect_stopss(sbb_oneday_english_h)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:

trips_collected_stops = stcweekdays.groupby("trip_id")\
            .agg(F.concat_ws("-", F.collect_list(splt(stcweekdays.stop_id))).alias("stop_ids")\
            ,F.concat_ws("-", F.collect_list(stcweekdays.departure_time)).alias("departures")).sort("stop_ids")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
trips_collected_stops_short = trips_collected_stops.filter((F.length("stop_ids")<=len("8502187-8502277-8502278"))&(F.length("stop_ids")>len("8502208")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
@F.udf(T.StringType())
def sample(sbb_stops):
    try:
        sbb_stops2 = "%"+ sbb_stops[10:-10] + "%"
        return sbb_stops2
    except:
        return sbb_stops
    
@F.udf(T.StringType())
def sample_short(sbb_stops):
    try:
        sbb_stops2 = "%"+ sbb_stops + "%"
        return sbb_stops2
    except:
        return sbb_stops   
    
@F.udf(T.StringType())
def sample_time(sbb_departure):
    return sbb_departure[:4]+"%"

#trips_collected_stops = trips_collected_stops.withColumn("sample_stop_ids",sample("stop_ids")).withColumn("sample_departures",sample_time("departures"))
trips_collected_stops_short = trips_collected_stops_short.withColumn("sample_stop_ids",sample_short("stop_ids")).withColumn("sample_departures",sample_time("departures"))



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
sbb_collected_stops.cache()
sbb_collected_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20120

In [58]:
sbb_collected_stops.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+--------------------+
|        trip_id|            stop_ids|          departures|
+---------------+--------------------+--------------------+
|  85:31:613:000|8502187-8502277-8...|07:01-07:02-07:04...|
|85:11:20484:001|8502208-8502209-8...|21:45-21:47-21:51...|
|85:11:20482:001|8502208-8502209-8...|21:15-21:17-21:21...|
|85:11:20480:001|8502208-8502209-8...|20:45-20:47-20:51...|
|85:11:20432:001|8502208-8502209-8...|08:45-08:47-08:51...|
|85:11:20440:001|8502208-8502209-8...|10:45-10:47-10:51...|
|85:11:20450:001|8502208-8502209-8...|13:15-13:17-13:21...|
|85:11:20438:001|8502208-8502209-8...|10:15-10:17-10:21...|
|85:11:20452:001|8502208-8502209-8...|13:45-13:47-13:51...|
|85:11:20448:001|8502208-8502209-8...|12:45-12:47-12:51...|
|85:11:20456:001|8502208-8502209-8...|14:45-14:47-14:51...|
|85:11:20442:001|8502208-8502209-8...|11:15-11:17-11:21...|
|85:11:20460:001|8502208-8502209-8...|15:45-15:47-15:51...|
|85:11:20454:001|8502208-8502209-8...|14

In [ ]:
trips_collected_stops_short = trips_collected_stops_short.

In [59]:
def match_trips(sbb_collected_stops,trips_collected_stops):
    
    
    sameStops = (sbb_collected_stops.stop_ids == trips_collected_stops.stop_ids)
    sameDepartures =( sbb_collected_stops.departures == trips_collected_stops.departures)

    trips_collected_stops = trips_collected_stops.alias("mts")
    sbb_collected_stops = sbb_collected_stops.alias("sbbs")
    matched_trips = sbb_collected_stops.join(trips_collected_stops,F.expr("sbbs.stop_ids LIKE mts.sample_stop_ids AND sbbs.departures LIKE mts.sample_departures"))
    return matched_trips

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
matched_trips = match_trips(sbb_collected_stops,trips_collected_stops_short)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
matched_trips.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string, sample_stop_ids: string, sample_departures: string]

In [62]:
matched_trips = matched_trips.select(F.col("sbbs.trip_id").alias("sbb_trip_id"),\
                    F.col("sbbs.stop_ids").alias("sbb_stop_ids"),\
                    F.col("sbbs.departures").alias("sbb_departures"),\
                    F.col("mts.trip_id").alias("trips_trip_id"),\
                    F.col("mts.stop_ids").alias("trips_stop_ids"),\
                    F.col("mts.departures").alias("trips_departures"),\
                    "sample_stop_ids","sample_departures" )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
matched_trips.write.parquet("user/boecuego/matched_trips2_shorts.parquet")

matched_trips = spark.read.parquet("user/boecuego/matched_trips2_shorts.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
matched_trips_shorts  = spark.read.parquet("user/boecuego/matched_trips2_shorts.parquet")
matched_trips = spark.read.parquet("user/boecuego/matched_trips2.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
matched_trips_shorts.filter("trips_trip_id == '32.TA.80-159-Y-j19-1.8.H'").show(20,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+------------------------+---------------+----------------+-----------------+-----------------+
|sbb_trip_id    |sbb_stop_ids                                                                                           |sbb_departures                                                               |trips_trip_id           |trips_stop_ids |trips_departures|sample_stop_ids  |sample_departures|
+---------------+-------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+------------------------+---------------+----------------+-----------------+-----------------+
|85:11:18844:001|8503204-8503203-8503202-8503201-8503200-8503009-8503010-8503011-8503000-8503006-8503129-8503306-85033

In [ ]:
matched_trips_shorts.select("")

In [30]:
matched_trips.filter(" LENGTH(sample_stop_ids) > 5  ").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

25577

In [31]:
matched_trips.filter("sample_stop_ids != '%%' AND LENGTH(sample_stop_ids) > 5  ").select("trips_trip_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20646

In [45]:
matched_trips.filter("trips_trip_id == '32.TA.80-159-Y-j19-1.8.H'").show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------+---------------+----------------+---------------+-----------------+
|sbb_trip_id          |sbb_stop_ids                                                                                                                                                                                                           |sbb_departures                                                                                                                                                   |trips_trip_id           |trips_stop_ids |trips_departures|sample_stop_ids|sample_departures|
+-----------

In [33]:
timematch = matched_trips.withColumn("exactTime",F.when(F.substring("sbb_departures",1,5) == F.substring("trips_departures",1,5),True ).otherwise(False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
timematch.columns[:-3]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['sbb_trip_id', 'sbb_stop_ids', 'sbb_departures', 'trips_trip_id', 'trips_stop_ids', 'trips_departures']

In [41]:
timematch.filter(" LENGTH(sample_stop_ids) > 5  ")\
.filter("exactTime == true")\
.select(*timematch.columns[:-3]).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+
|sbb_trip_id          |sbb_stop_ids                                                                                                                                   |sbb_departures                                                                                             |trips_trip_id           |trips_stop_ids                                                                                                                                 |tri

In [15]:
matched_trips.filter("sample_stop_ids != '%%' AND LENGTH(sample_stop_ids) > 5  ").show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------------------------+-----------------------+-------------------------+-----------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+---------------+-----------------+
|sbb_trip_id    |sbb_stop_ids                   |sbb_departures         |trips_trip_id            |trips_stop_ids                                                                                 |trips_departures                                                       |sample_stop_ids|sample_departures|
+---------------+-------------------------------+-----------------------+-------------------------+-----------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+---------------+-----------------+
|85:11:14023:002|8503200-8503201-8503202-8503203|07:02-07:05-07:08-07:10|616.TA.26-8-A-j19-1.3

In [ ]:
print("OK")

In [ ]:
# match_trips(sbb_collected_stops,trips_collected_stops

In [67]:
sbb_p1 = collect_stopss(sbb_oneday_english_h.withColumn("departure_time",F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))+60,"HH:mm")))
sbb_p2 = collect_stopss(sbb_oneday_english_h.withColumn("departure_time",F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))+2*60,"HH:mm")))
sbb_p3 = collect_stopss(sbb_oneday_english_h.withColumn("departure_time",F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))+3*60,"HH:mm")))
sbb_m1 = collect_stopss(sbb_oneday_english_h.withColumn("departure_time",F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))-60,"HH:mm")))
sbb_m2 = collect_stopss(sbb_oneday_english_h.withColumn("departure_time",F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))-2*60,"HH:mm")))
sbb_m3 = collect_stopss(sbb_oneday_english_h.withColumn("departure_time",F.from_unixtime(F.unix_timestamp(F.col("departure_time").cast("timestamp"))-3*60,"HH:mm")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
matched_trips_p1 = match_trips(sbb_p1,trips_collected_stops)
matched_trips_p2 = match_trips(sbb_p2,trips_collected_stops)
matched_trips_p3 = match_trips(sbb_p3,trips_collected_stops)
matched_trips_m1 = match_trips(sbb_m1,trips_collected_stops)
matched_trips_m2 = match_trips(sbb_m2,trips_collected_stops)
matched_trips_m3 = match_trips(sbb_m3,trips_collected_stops)
matcheds  =[matched_trips_p1,matched_trips_p2,matched_trips_p3,matched_trips_m1,matched_trips_m2,matched_trips_m3]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [69]:
for matched in matcheds:
    matched.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]
DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]
DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]
DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]
DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]
DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]

In [70]:
for matched in matcheds:
    print(matched.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16
31
32
39
8
10

In [28]:
matched_trips.cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trip_id: string, stop_ids: string, departures: string, trip_id: string, stop_ids: string, departures: string]

In [29]:
matched_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1141

In [50]:
trips_collected_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27752

In [42]:
matched_trips.filter(trips_collected_stops.stop_ids.like("8590314-8594307-8594310-8594304")).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------------------+-----------------------+----------------------+-------------------------------+-----------------------+
|trip_id       |stop_ids                       |departures             |trip_id               |stop_ids                       |departures             |
+--------------+-------------------------------+-----------------------+----------------------+-------------------------------+-----------------------+
|85:886:45603-0|8590314-8594307-8594310-8594304|19:07-19:09-19:10-19:11|55.TA.1-11-B-j19-1.2.H|8590314-8594307-8594310-8594304|19:07-19:09-19:10-19:11|
|85:886:45495-0|8590314-8594307-8594310-8594304|10:07-10:09-10:10-10:11|37.TA.1-11-B-j19-1.2.H|8590314-8594307-8594310-8594304|10:07-10:09-10:10-10:11|
|85:886:45555-0|8590314-8594307-8594310-8594304|15:07-15:09-15:10-15:11|47.TA.1-11-B-j19-1.2.H|8590314-8594307-8594310-8594304|15:07-15:09-15:10-15:11|
|85:886:45591-0|8590314-8594307-8594310-8594304|18:07-18:09-18:10-18:11|53.TA.1-11-B-j19

In [91]:
stops = "8591264-8591181-8591349-8591343-8591420-8591393-8591276-8591406-8591244-8591255-8591229-8591362-8591433-8591173-8591123-8591220-8591239-8576193-8591105-8591093-8591299-8591369-8591381-8591427"
stops_like = stops.replace("-","%")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
stops_like = "%"+stops_like+"%"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
sbb_collected_stops.filter(F.col("stop_ids").like(stops_like)).filter(F.col("departures").like("12:1%")).show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|trip_id              |stop_ids                                                                                                                                                                                                                                       |departures                                                                                                                                                                         |
+---------------------+-----------------------------------------------------------------------------------------

In [74]:
trips_collected_stops.filter("trip_id == '3192.TA.26-9-B-j19-1.25.H'").show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|trip_id                  |stop_ids                                                                                                                                                                                                                                                       |departures                                                                                                                                                                                     |
+-------------------------+---------------------

In [98]:
x = "8587652-8591263-8591264-8591181-8591349-8591343-8591420-8591393-8591276-8591406-8591244-8591255-8591229-8591362-8591433-8591173-8591123-8591220-8591239-8576193-8591105-8591093-8591299-8591369-8591381-8591427-8573710-8591341-8502572-8591390"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
y = "8587652-8591263-8591264-8591181-8591349-8591343-8591420-8591393-8591276-8591406-8591244-8591255-8591229-8591362-8591433-8591173-8591123-8591220-8591239-8576193-8591105-8591093-8591299-8591369-8591381-8591427-8573710-8591341-8502572-8591390"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
x == y

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [72]:

matched_trips.filter(trips_collected_stops.trip_id == "3192.TA.26-9-B-j19-1.25.H").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+----------+-------+--------+----------+
|trip_id|stop_ids|departures|trip_id|stop_ids|departures|
+-------+--------+----------+-------+--------+----------+
+-------+--------+----------+-------+--------+----------+

In [ ]:
matched_trips.drop(trips_collected_stops.stop_ids)

In [71]:
matched_trips.select(trips_collected_stops.stop_ids).distinct().show(40,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|stop_ids                                                                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|8503512-8503509-8503001-8503020-8503000-8503003-8503147                                                                                                        |
|8503313-8503312-8503311-8503310-8503006-8503020-8503000                                                                                                        |
|8503129-8503306-8503305                                                                                                                                        |
|8594304-8594310-8594307-859

In [134]:
matched_trips.groupBy(sbb_collected_stops.trip_id,trips_collected_stops.trip_id).count().filter("count >1").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

### Number of unique trip_ids

In [130]:
sbb_collected_stops.count() #HW1 data unique trips on date 14.05.2019

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24318

In [131]:
trips_collected_stops.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

27752

In [132]:
matched_trips.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

520

In [ ]:
sbb_oneday_english.select("trip_id").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
matched_trips.take(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(trip_id=u'85:801:107754-01504-1', stop_ids=u'8573237-8573236-8573263-8573262-8573261-8503871-8503782-8573260', departures=u'15:31-15:33-15:35-15:36-15:37-15:40-15:42-15:43', trip_id=u'331.TA.26-525-j19-1.4.R', stop_ids=u'8573237-8573236-8573263-8573262-8573261-8503871-8503782-8573260', departures=u'15:31-15:33-15:35-15:36-15:37-15:40-15:42-15:43'), Row(trip_id=u'85:801:107754-01504-1', stop_ids=u'8573237-8573236-8573263-8573262-8573261-8503871-8503782-8573260', departures=u'15:31-15:33-15:35-15:36-15:37-15:40-15:42-15:43', trip_id=u'96.TA.26-525-j19-1.3.R', stop_ids=u'8573237-8573236-8573263-8573262-8573261-8503871-8503782-8573260', departures=u'15:31-15:33-15:35-15:36-15:37-15:40-15:42-15:43'), Row(trip_id=u'85:801:107754-01504-1', stop_ids=u'8573237-8573236-8573263-8573262-8573261-8503871-8503782-8573260', departures=u'15:31-15:33-15:35-15:36-15:37-15:40-15:42-15:43', trip_id=u'220.TA.26-525-j19-1.4.R', stop_ids=u'8573237-8573236-8573263-8573262-8573261-8503871-8503782-8573260',

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
stops_joined_df = spark.read.parquet("user/boecuego/stops_joined_df.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
stops_joined_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+-------------+--------------+
|             trip_id|arrival_time|departure_time|        stop_id|stop_sequence|         stop_name|        stop_lat|        stop_lon|location_type|parent_station|
+--------------------+------------+--------------+---------------+-------------+------------------+----------------+----------------+-------------+--------------+
|5.TA.1-1-A-j19-1.3.H|    02:42:00|      02:42:00|    8503305:0:3|            2|        Effretikon|  47.42593043029|8.68666388288076|             |      8503305P|
|5.TA.1-1-A-j19-1.3.H|    02:46:00|      02:46:00|    8503306:0:3|            3|         Dietlikon|47.4203145327083|8.61925430395105|             |      8503306P|
|5.TA.1-1-A-j19-1.3.H|    02:50:00|      02:50:00|    8503147:0:1|            4|         Stettbach| 47.397334167601|8.59613166853459|             |      8503147P|
|5.TA.1-1-A-j19-1.3.H|

In [6]:
df_grouped = stops_joined_df.groupBy("trip_id").agg(F.collect_list("stop_id").alias("stop_ids"),F.collect_list("stop_sequence").alias("stop_sequences"),\
                                      F.collect_list("arrival_time").alias("arrival_times"),\
                                      F.collect_list("departure_time").alias("departure_times"),\
                                      )                                    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
import itertools
@F.udf(T.ArrayType(
       T.ArrayType(T.StructType([
    T.StructField("pair1", T.StringType(), False),
    T.StructField("pair2", T.StringType(), False)
]))))
def pairs(stop_ids,stop_sequences,arrival_times,departure_times):
    stop_id_pairs = list(itertools.combinations(stop_ids,2))
    stop_sequence_pairs = list(itertools.combinations(stop_sequences,2))
    arrival_time_pairs = list(itertools.combinations(arrival_times,2))
    departure_time_pairs = list(itertools.combinations(departure_times,2))
    return [stop_id_pairs,stop_sequence_pairs,arrival_time_pairs,departure_time_pairs]
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_grouped.cache()
df_grouped.take(1)
df_grouped.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

173184

In [9]:
df_pairs = df_grouped.withColumn("all_pairs",pairs("stop_ids","stop_sequences","arrival_times","departure_times"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_pairs.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             trip_id|            stop_ids|      stop_sequences|       arrival_times|     departure_times|           all_pairs|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142...|[1, 2, 3, 4, 5, 6...|[28:45:00, 28:46:...|[28:45:00, 28:46:...|[[[8591315, 85911...|
|1.TA.63-138-Y-j19...|      [8503000:0:16]|                 [1]|          [18:22:00]|          [18:22:00]|    [[], [], [], []]|
|10.TA.80-55-Y-j19...|[8503202:0:5, 850...|            [10, 12]|[07:38:00, 07:48:00]|[07:39:00, 07:48:00]|[[[8503202:0:5, 8...|
|1000.TA.26-768-j1...|[8580449, 8591063...|[1, 2, 3, 4, 5, 6...|[22:45:00, 22:46:...|[22:45:00, 22:46:...|[[[8580449, 85910...|
|1005.TA.26-131-j1...|[8503855:0:F, 858...|[1, 2, 3, 4, 5, 6...|[16:14:00, 16:15:...|[16:14:00, 16:15:..

In [12]:

df_pairs_sep = df_pairs.withColumn("stop_id_pairs",F.col("all_pairs")[0])\
                        .withColumn("stop_sequence_pairs",F.col("all_pairs")[1])\
                        .withColumn("arrival_time_pairs",F.col("all_pairs")[2])\
                        .withColumn("departure_time_pairs",F.col("all_pairs")[3])


df_pairs_sep.select("trip_id",F.explode("stop_id_pairs")).withColumn("rown",F.monotonically_increasing_id()).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+----+
|             trip_id|               col|rown|
+--------------------+------------------+----+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142]|   0|
|1.TA.26-18-j19-1.1.H|[8591315, 8530811]|   1|
|1.TA.26-18-j19-1.1.H|[8591315, 8591104]|   2|
|1.TA.26-18-j19-1.1.H|[8591315, 8591429]|   3|
|1.TA.26-18-j19-1.1.H|[8591315, 8591180]|   4|
|1.TA.26-18-j19-1.1.H|[8591315, 8530812]|   5|
|1.TA.26-18-j19-1.1.H|[8591315, 8591364]|   6|
|1.TA.26-18-j19-1.1.H|[8591315, 8530813]|   7|
|1.TA.26-18-j19-1.1.H|[8591142, 8530811]|   8|
|1.TA.26-18-j19-1.1.H|[8591142, 8591104]|   9|
|1.TA.26-18-j19-1.1.H|[8591142, 8591429]|  10|
|1.TA.26-18-j19-1.1.H|[8591142, 8591180]|  11|
|1.TA.26-18-j19-1.1.H|[8591142, 8530812]|  12|
|1.TA.26-18-j19-1.1.H|[8591142, 8591364]|  13|
|1.TA.26-18-j19-1.1.H|[8591142, 8530813]|  14|
|1.TA.26-18-j19-1.1.H|[8530811, 8591104]|  15|
|1.TA.26-18-j19-1.1.H|[8530811, 8591429]|  16|
|1.TA.26-18-j19-1.1.H|[8530811, 8591180]|  17|
|1.TA.26-18-j

In [21]:
sti = df_pairs_sep.select("trip_id",F.explode("stop_id_pairs").alias("stop_id_pair")).withColumn("rown",F.monotonically_increasing_id()).alias("sti")
ssp = df_pairs_sep.select(F.explode("stop_sequence_pairs").alias("stop_sequence_pair"),F.monotonically_increasing_id().alias("rown")).alias("ssp")
atp = df_pairs_sep.select(F.explode("arrival_time_pairs").alias("arrival_time_pair"),F.monotonically_increasing_id().alias("rown")).alias("atp")
dtp = df_pairs_sep.select(F.explode("departure_time_pairs").alias("departure_time_pair"),F.monotonically_increasing_id().alias("rown")).alias("dtp")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
sti.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+----+
|             trip_id|      stop_id_pair|rown|
+--------------------+------------------+----+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142]|   0|
|1.TA.26-18-j19-1.1.H|[8591315, 8530811]|   1|
|1.TA.26-18-j19-1.1.H|[8591315, 8591104]|   2|
|1.TA.26-18-j19-1.1.H|[8591315, 8591429]|   3|
|1.TA.26-18-j19-1.1.H|[8591315, 8591180]|   4|
|1.TA.26-18-j19-1.1.H|[8591315, 8530812]|   5|
|1.TA.26-18-j19-1.1.H|[8591315, 8591364]|   6|
|1.TA.26-18-j19-1.1.H|[8591315, 8530813]|   7|
|1.TA.26-18-j19-1.1.H|[8591142, 8530811]|   8|
|1.TA.26-18-j19-1.1.H|[8591142, 8591104]|   9|
|1.TA.26-18-j19-1.1.H|[8591142, 8591429]|  10|
|1.TA.26-18-j19-1.1.H|[8591142, 8591180]|  11|
|1.TA.26-18-j19-1.1.H|[8591142, 8530812]|  12|
|1.TA.26-18-j19-1.1.H|[8591142, 8591364]|  13|
|1.TA.26-18-j19-1.1.H|[8591142, 8530813]|  14|
|1.TA.26-18-j19-1.1.H|[8530811, 8591104]|  15|
|1.TA.26-18-j19-1.1.H|[8530811, 8591429]|  16|
|1.TA.26-18-j19-1.1.H|[8530811, 8591180]|  17|
|1.TA.26-18-j

In [24]:
ssp.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----+
|stop_sequence_pair|rown|
+------------------+----+
|            [1, 2]|   0|
|            [1, 3]|   1|
|            [1, 4]|   2|
|            [1, 5]|   3|
|            [1, 6]|   4|
|            [1, 7]|   5|
|            [1, 8]|   6|
|            [1, 9]|   7|
|            [2, 3]|   8|
|            [2, 4]|   9|
|            [2, 5]|  10|
|            [2, 6]|  11|
|            [2, 7]|  12|
|            [2, 8]|  13|
|            [2, 9]|  14|
|            [3, 4]|  15|
|            [3, 5]|  16|
|            [3, 6]|  17|
|            [3, 7]|  18|
|            [3, 8]|  19|
+------------------+----+
only showing top 20 rows

In [26]:
ssp.cache()
ssp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20584205

In [ ]:
sti.cache()
ssp.cache()
atp.cache()
dtp.cache()


In [ ]:
sti.count()
ssp.count()
atp.count()
dtp.count()

In [48]:
joined = sti.join(ssp,sti.rown == ssp.rown).drop(ssp.rown).join(atp,sti.rown == atp.rown).drop(atp.rown).join(dtp,sti.rown == dtp.rown).drop(dtp.rown)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
sti.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+----+
|             trip_id|      stop_id_pair|rown|
+--------------------+------------------+----+
|1.TA.26-18-j19-1.1.H|[8591315, 8591142]|   0|
|1.TA.26-18-j19-1.1.H|[8591315, 8530811]|   1|
|1.TA.26-18-j19-1.1.H|[8591315, 8591104]|   2|
|1.TA.26-18-j19-1.1.H|[8591315, 8591429]|   3|
|1.TA.26-18-j19-1.1.H|[8591315, 8591180]|   4|
|1.TA.26-18-j19-1.1.H|[8591315, 8530812]|   5|
|1.TA.26-18-j19-1.1.H|[8591315, 8591364]|   6|
|1.TA.26-18-j19-1.1.H|[8591315, 8530813]|   7|
|1.TA.26-18-j19-1.1.H|[8591142, 8530811]|   8|
|1.TA.26-18-j19-1.1.H|[8591142, 8591104]|   9|
|1.TA.26-18-j19-1.1.H|[8591142, 8591429]|  10|
|1.TA.26-18-j19-1.1.H|[8591142, 8591180]|  11|
|1.TA.26-18-j19-1.1.H|[8591142, 8530812]|  12|
|1.TA.26-18-j19-1.1.H|[8591142, 8591364]|  13|
|1.TA.26-18-j19-1.1.H|[8591142, 8530813]|  14|
|1.TA.26-18-j19-1.1.H|[8530811, 8591104]|  15|
|1.TA.26-18-j19-1.1.H|[8530811, 8591429]|  16|
|1.TA.26-18-j19-1.1.H|[8530811, 8591180]|  17|
|1.TA.26-18-j

In [49]:
#joined.write.mode("overwrite").parquet("user/boecuego/df_all_stop_connections.parquet")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
joined = spark.read.parquet("user/boecuego/df_all_stop_connections.parquet")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
stop_times_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stop_times")
stops_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/stops")
trips_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/trips")
calendar_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/calendar")
routes_df = spark.read.orc("hdfs:///data/sbb/timetables/orc/routes")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_service  =joined.join(trips_df,trips_df.trip_id ==joined.trip_id  )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
df_days = df_service.join(calendar_df,calendar_df.service_id == df_service.service_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
df_weekdays = df_days.filter("monday == 'true' AND tuesday == 'true' AND wednesday == 'true' AND thursday == 'true' AND friday == 'true' ")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
df_weekdays

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_weekdays' is not defined
Traceback (most recent call last):
NameError: name 'df_weekdays' is not defined



In [56]:
cols = joined.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_weekdays.drop(trips_df.trip_id)

In [57]:
#joined_filtered  = df_weekdays.select(*cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u"Reference 'trip_id' is ambiguous, could be: trip_id, trip_id.;"
Traceback (most recent call last):
  File "/hdata/sde/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2044/container_e06_1589299642358_2044_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 1202, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "/hdata/sde/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2044/container_e06_1589299642358_2044_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sde/hadoop/yarn/local/usercache/ebouille/appcache/application_1589299642358_2044/container_e06_1589299642358_2044_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u"Reference 'trip_id' is ambiguous, could be: trip_id, trip_id.;"



In [66]:
joined_filtered

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trip_id: string, stop_id_pair: struct<pair1:string,pair2:string>, rown: bigint, stop_sequence_pair: struct<pair1:string,pair2:string>, arrival_time_pair: struct<pair1:string,pair2:string>, departure_time_pair: struct<pair1:string,pair2:string>]

In [59]:
#joined_filtered.write.mode("overwrite").parquet("user/boecuego/df_all_stop_connections_filtered.parquet")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'joined_filtered' is not defined
Traceback (most recent call last):
NameError: name 'joined_filtered' is not defined



In [ ]:
joined_filtered = spark.read.parquet("user/boecuego/df_all_stop_connections_filtered.parquet")

In [67]:
joined_filtered.show() 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----+------------------+--------------------+--------------------+
|             trip_id|        stop_id_pair| rown|stop_sequence_pair|   arrival_time_pair| departure_time_pair|
+--------------------+--------------------+-----+------------------+--------------------+--------------------+
|1087.TA.26-5-B-j1...|  [8591366, 8591415]| 2637|            [3, 4]|[15:45:00, 15:46:00]|[15:45:00, 15:46:00]|
|1087.TA.26-5-B-j1...|  [8591303, 8591230]| 2701|          [12, 14]|[16:00:00, 16:04:00]|[16:00:00, 16:04:00]|
|1099.TA.26-142-j1...|  [8590825, 8573167]| 3197|            [4, 7]|[17:35:00, 17:40:00]|[17:35:00, 17:40:00]|
|111.TA.79-736-j19...|  [8591031, 8580301]| 4154|            [1, 3]|[18:39:00, 18:42:00]|[18:39:00, 18:42:00]|
|1141.TA.26-5-B-j1...|  [8591366, 8591220]| 5499|           [3, 11]|[10:37:00, 10:51:00]|[10:37:00, 10:51:00]|
|1141.TA.26-5-B-j1...|  [8591366, 8591303]| 5500|           [3, 12]|[10:37:00, 10:53:00]|[10:37:00, 10:53:00]|
|

In [68]:
joineds = joined_filtered.sort("rown").select("trip_id",F.col("stop_id_pair").pair1.alias("stop_id1")\
                        ,F.col("stop_id_pair").pair2.alias("stop_id2")\
                        ,F.col("stop_sequence_pair").pair1.alias("stop_sequence1")\
                        ,F.col("stop_sequence_pair").pair2.alias("stop_sequence2")\
                        ,F.col("arrival_time_pair").pair1.alias("arrival_time1")\
                        ,F.col("arrival_time_pair").pair2.alias("arrival_time2")\
                        ,F.col("departure_time_pair").pair1.alias("departure_time1")\
                        ,F.col("departure_time_pair").pair2.alias("departure_time2"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
joineds.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+--------+--------------+--------------+-------------+-------------+---------------+---------------+
|             trip_id|   stop_id1|stop_id2|stop_sequence1|stop_sequence2|arrival_time1|arrival_time2|departure_time1|departure_time2|
+--------------------+-----------+--------+--------------+--------------+-------------+-------------+---------------+---------------+
|1005.TA.26-131-j1...|8503855:0:F| 8589111|             1|             2|     16:14:00|     16:15:00|       16:14:00|       16:15:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573553|             1|             3|     16:14:00|     16:16:00|       16:14:00|       16:16:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573554|             1|             4|     16:14:00|     16:18:00|       16:14:00|       16:18:00|
|1005.TA.26-131-j1...|8503855:0:F| 8573555|             1|             5|     16:14:00|     16:19:00|       16:14:00|       16:19:00|
|1005.TA.26-131-j1...|8503855:0:F| 8588985|             1|    

In [71]:
joineds.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3291068

In [ ]:
df_pairs_sep.cache()
df_pairs_sep.count()

In [ ]:
df_pairs_sep.select("row_num"F.explode("stop_id_pairs"),"stop_sequence_pairs","row_num").show()

In [54]:
from pyspark.sql.window import Window
w = Window().orderBy(F.lit('A'))
df = df.withColumn("row_num", F.row_number().over(w))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- stop_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_sequences: array (nullable = true)
 |    |-- element: short (containsNull = true)
 |-- arrival_times: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- departure_times: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- all_pairs: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- pair1: string (nullable = false)
 |    |    |    |-- pair2: string (nullable = false)
 |-- stop_id_pairs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- pair1: string (nullable = false)
 |    |    |-- pair2: string (nullable = false)
 |-- stop_sequence_pairs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- pair1: string (nullable = false)
 |    |    

In [55]:
sttype = T.ArrayType(T.StructType([
    T.StructField("pair1", T.StringType(), False),
    T.StructField("pair2", T.StringType(), False)
]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
@F.udf(T.ArrayType(T.StructType([T.StructField("stop_id_pairs", sttype),
                                    T.StructField("stop_sequence_pairs", sttype),
                                    T.StructField("arrival_time_pairs", sttype),
                                    T.StructField("departure_time_pairs", sttype)])))
def combine(a,b,c,d):
      return list(zip(a,b,c,d))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_pairs_sep_combine = df_pairs_sep.limit(2).withColumn("new", combine("stop_id_pairs", "stop_sequence_pairs","arrival_time_pairs","departure_time_pairs"))

In [58]:
df_pairs_sep_combine.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [46]:
zip([1,2],[2,3],[4,6])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, 2, 4), (2, 3, 6)]

In [19]:
list(itertools.combinations([1,2,3,4],2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [ ]:
itertools.combinations([1,2,3,4],2)

In [ ]:


df_grouped.rdd.map(lambda x:x[0]).take(1)